In [ ]:
using MXNet

In [ ]:
data = mx.Variable(:data)

In [ ]:
fc1  = mx.FullyConnected(data, name=:fc1, num_hidden=128)
act1 = mx.Activation(fc1, name=:relu1, act_type=:relu)
fc2  = mx.FullyConnected(act1, name=:fc2, num_hidden=64)
act2 = mx.Activation(fc2, name=:relu2, act_type=:relu)
fc3  = mx.FullyConnected(act2, name=:fc3, num_hidden=10)

In [ ]:
mlp  = mx.SoftmaxOutput(fc2, name=:softmax)

In [ ]:
mlp = @mx.chain mx.Variable(:data)             =>
  mx.FullyConnected(name=:fc1, num_hidden=128) =>
  mx.Activation(name=:relu1, act_type=:relu)   =>
  mx.FullyConnected(name=:fc2, num_hidden=64)  =>
  mx.Activation(name=:relu2, act_type=:relu)   =>
  mx.FullyConnected(name=:fc3, num_hidden=10)  =>
  mx.SoftmaxOutput(name=:softmax)

In [ ]:
batch_size = 100
include("/home/jens/.julia/v0.6/MXNet/examples/mnist/mnist-data.jl")
train_provider, eval_provider = get_mnist_providers(batch_size; flat = false)

In [ ]:
model = mx.FeedForward(mlp, context=mx.cpu())

In [ ]:
optimizer = mx.ADAM()

In [ ]:
@time mx.fit(model, optimizer, train_provider, n_epoch=10, verbosity = 0)

In [ ]:
# first conv
conv1 = @mx.chain mx.Convolution(data, kernel=(5,5), num_filter=20)  =>
                  mx.Activation(act_type=:tanh) =>
                  mx.Pooling(pool_type=:max, kernel=(2,2), stride=(2,2))

# second conv
conv2 = @mx.chain mx.Convolution(conv1, kernel=(5,5), num_filter=50) =>
                  mx.Activation(act_type=:tanh) =>
                  mx.Pooling(pool_type=:max, kernel=(2,2), stride=(2,2))

In [ ]:
# first fully-connected
fc1   = @mx.chain mx.Flatten(conv2) =>
                  mx.FullyConnected(num_hidden=500) =>
                  mx.Activation(act_type=:tanh)

# second fully-connected
fc2   = mx.FullyConnected(fc1, num_hidden=10)

# softmax loss
lenet = mx.SoftmaxOutput(fc2, name=:softmax)

In [ ]:
# fit model
model = mx.FeedForward(lenet, context=mx.cpu())

# optimizer
optimizer = mx.ADAM()

In [ ]:
# fit parameters
@time mx.fit(model, optimizer, train_provider, n_epoch=2, eval_data=eval_provider, verbosity = 2)